In [1]:
1+1

2

In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import BaseMessage
from langchain_core.agents import AgentAction, AgentFinish
import operator

import logging

logging.basicConfig(level=logging.DEBUG)

# 환경 변수
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")


# AgentState 정의 (올바른 타입 지정)
class AgentState(TypedDict):
    input: str
    chat_history: Annotated[List[BaseMessage], operator.add]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[List[tuple], operator.add]
    output: str  # 최종 출력용 필드 추가


# 기존 도구 함수들
def add_two(x: str) -> str:
    try:
        return str(float(x) + 2)
    except Exception:
        raise ValueError("숫자만 입력해 주세요.")


def population_lookup(city: str) -> str:
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    # 대소문자 구분 없이 검색하도록 수정
    city_lower = city.strip().lower()
    for key, value in data.items():
        if key.lower() == city_lower:
            return value
    raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# 도구 및 LLM 설정
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 이름 입력 시 인구 정보를 반환합니다.",
    ),
]

llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0.1, openai_api_key=OPEN_API_KEY, verbose=True
)

# ReAct 프롬프트 설정
react_prompt = PromptTemplate.from_template(
    """
당신은 도움이 되는 어시스턴트입니다. 주어진 도구를 사용해서 질문에 답하세요.

사용 가능한 도구:
{tools}

도구 이름: {tool_names}

다음 형식을 사용하세요:

Question: 답변해야 할 입력 질문
Thought: 무엇을 해야 할지 생각해보세요
Action: 수행할 액션, [{tool_names}] 중 하나여야 합니다
Action Input: 액션에 대한 입력
Observation: 액션의 결과
... (이 Thought/Action/Action Input/Observation을 필요한 만큼 반복)
Thought: 이제 최종 답변을 알았습니다
Final Answer: 원래 입력 질문에 대한 최종 답변

시작하세요!

Question: {input}
Thought: {agent_scratchpad}
"""
)

# ReAct Agent와 AgentExecutor 생성
react_agent = create_react_agent(llm, tools, react_prompt)
agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,  # Agent 실행 과정 출력
    return_intermediate_steps=True,  # 중간 단계 반환
    max_iterations=10,  # 디버깅용 최대 반복 제한
)


# Agent 노드 함수 (수정됨)
def agent_node(state: AgentState) -> AgentState:
    try:
        # AgentExecutor 사용하여 실행
        result = agent_executor.invoke(
            {
                "input": state["input"],
                "chat_history": state.get("chat_history", []),
                "intermediate_steps": state.get("intermediate_steps", []),
            }
        )

        print(f"✅ [DEBUG] Agent 실행 성공")
        print(f"📊 [DEBUG] 중간 단계 수: {len(result.get('intermediate_steps', []))}")

        return {
            "input": state["input"],
            "chat_history": state.get("chat_history", []),
            "agent_outcome": None,  # AgentExecutor는 이미 완료된 결과를 반환
            "intermediate_steps": result.get("intermediate_steps", []),
            "output": result["output"],  # 최종 결과를 output 필드에 저장
        }
    except Exception as e:
        print(f"❌ [DEBUG] Agent 실행 실패: {str(e)}")
        print(f"🔄 [DEBUG] Fallback 모드로 전환")

        # Fallback 처리
        response = llm.invoke(f"다음 질문에 답해주세요: {state['input']}")
        fallback_result = f"🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n{response.content}"

        return {
            "input": state["input"],
            "chat_history": state.get("chat_history", []),
            "agent_outcome": None,
            "intermediate_steps": state.get("intermediate_steps", []),
            "output": fallback_result,
        }


# StateGraph 생성
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)

# 시작점과 종료점 설정
graph.set_entry_point("agent")
graph.add_edge("agent", END)

# 그래프 컴파일
smart_agent = graph.compile(debug=True)

# 테스트
if __name__ == "__main__":
    # 도구에서 찾을 수 있는 정보
    print("=== 도구에서 찾을 수 있는 정보 ===")
    result1 = smart_agent.invoke(
        {
            "input": "서울의 인구는?",
            "chat_history": [],
            "agent_outcome": None,
            "intermediate_steps": [],
            "output": "",
        }
    )
    print(result1["output"])

    print("\n=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===")
    result2 = smart_agent.invoke(
        {
            "input": "대구의 인구는?",
            "chat_history": [],
            "agent_outcome": None,
            "intermediate_steps": [],
            "output": "",
        }
    )
    print(result2["output"])

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-cb95d90c-1834-491b-9cef-f4adb2b3c69b', 'json_data': {'messages': [{'content': '다음 질문에 답해주세요: 서울의 인구는?', 'role': 'user'}], 'model': 'gpt-4o-mini', 'stream': False, 'temperature': 0.1}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000002868ABFC5F0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000002868A66A060> server_hostname='api.openai.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncS

=== 도구에서 찾을 수 있는 정보 ===
[values] {'input': '서울의 인구는?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'output': ''}


> Entering new AgentExecutor chain...
❌ [DEBUG] Agent 실행 실패: RunnableAgent.plan() got multiple values for argument 'intermediate_steps'
🔄 [DEBUG] Fallback 모드로 전환


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Jul 2025 13:45:37 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'ailysfrontier'), (b'openai-processing-ms', b'1625'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'1628'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'199983'), (b'x-ratelimit-reset-requests', b'8.64s'), (b'x-ratelimit-reset-tokens', b'4ms'), (b'x-request-id', b'req_bc40a2cc68cffda59a2545aa2ca31690'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'959f105dff88ea15-ICN

[updates] {'agent': {'input': '서울의 인구는?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'output': '🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n2023년 기준으로 서울의 인구는 약 9백만 명 정도입니다. 하지만 인구는 지속적으로 변동할 수 있으므로, 최신 통계 자료를 확인하는 것이 좋습니다. 서울특별시청이나 통계청의 공식 웹사이트에서 최신 정보를 확인할 수 있습니다.'}}
[values] {'input': '서울의 인구는?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'output': '🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n2023년 기준으로 서울의 인구는 약 9백만 명 정도입니다. 하지만 인구는 지속적으로 변동할 수 있으므로, 최신 통계 자료를 확인하는 것이 좋습니다. 서울특별시청이나 통계청의 공식 웹사이트에서 최신 정보를 확인할 수 있습니다.'}
🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
2023년 기준으로 서울의 인구는 약 9백만 명 정도입니다. 하지만 인구는 지속적으로 변동할 수 있으므로, 최신 통계 자료를 확인하는 것이 좋습니다. 서울특별시청이나 통계청의 공식 웹사이트에서 최신 정보를 확인할 수 있습니다.

=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===
[values] {'input': '대구의 인구는?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'output': ''}


> Entering new AgentExecutor chain...
❌ [DEBUG] Agent 실행 실패: RunnableAgent.plan() got multiple values for argument 'intermediat

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Jul 2025 13:45:39 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'ailysfrontier'), (b'openai-processing-ms', b'1092'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'1097'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9998'), (b'x-ratelimit-remaining-tokens', b'199983'), (b'x-ratelimit-reset-requests', b'15.201s'), (b'x-ratelimit-reset-tokens', b'4ms'), (b'x-request-id', b'req_196f69a1216535d1d8bd054a5f1419bd'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'959f10697d31ea15-I

[updates] {'agent': {'input': '대구의 인구는?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'output': '🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구의 인구에 대한 정확한 수치는 대구광역시청이나 통계청의 공식 자료를 참고하시기 바랍니다.'}}
[values] {'input': '대구의 인구는?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': [], 'output': '🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구의 인구에 대한 정확한 수치는 대구광역시청이나 통계청의 공식 자료를 참고하시기 바랍니다.'}
🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구의 인구에 대한 정확한 수치는 대구광역시청이나 통계청의 공식 자료를 참고하시기 바랍니다.
